In [8]:
!pip install pyarrow==2 awswrangler

In [9]:
import os
import numpy as np
import pandas as pd
import sagemaker
import awswrangler as wr
from sklearn.model_selection import train_test_split
from sagemaker.pytorch import PyTorch
from sagemaker.tuner import (
    CategoricalParameter,
    HyperparameterTuner,
)

In [10]:
sagemaker_session = sagemaker.Session()

bucket = 'cgu-poc-sagemaker'
data_prefix = 'datasets/data-wrangler-feedbacks-2022-06-14T03-02-28'
model_prefix = 'models/feedbacks'

role = sagemaker.get_execution_role()

In [11]:
df = wr.s3.read_csv(f"s3://{bucket}/{data_prefix}/")
df.head()

,labels,comments
0,0,minha crítica não é em relação ao produto em si
1,1,recomendo a todos. a bomba submersa anauger é ...
2,0,o cabo é ridículamente curto. a chaleira tem q...
3,0,gostaria de saber se o colchão vem incluso? po...
4,0,não vale a pena. economia porca


In [12]:
train, test = train_test_split(df)
df.to_csv('./data/dataset.csv', index=False)
train.to_csv("./data/feedbacks_train.csv", index=False)
test.to_csv("./data/feedbacks_test.csv", index=False)

In [13]:
inputs_train = sagemaker_session.upload_data("./data/feedbacks_train.csv", bucket=bucket, key_prefix=f'{data_prefix}/train')
inputs_test = sagemaker_session.upload_data("./data/feedbacks_test.csv", bucket=bucket, key_prefix=f'{data_prefix}/test')

In [ ]:
# place to save model artifact
output_path = f"s3://{bucket}/{model_prefix}"

estimator = PyTorch(
    entry_point="train.py",
    source_dir="script",
    role=role,
    framework_version="1.10.0",
    py_version="py38",
    instance_count=2,  # this script only support distributed training for GPU instances.
    instance_type="ml.p3.8xlarge",
    # instance_type="local",
    output_path=output_path,
    hyperparameters={
        "batch-size": 16,
        "epochs": 3,
        "num_labels": 2,
        "backend": "gloo",
    },
    disable_profiler=True, # disable debugger
)
estimator.fit({"training": inputs_train, "testing": inputs_test})

INFO:sagemaker:Creating training-job with name: pytorch-training-2022-06-27-01-29-38-732


2022-06-27 01:29:38 Starting - Starting the training job......
2022-06-27 01:30:33 Starting - Preparing the instances for training.........
2022-06-27 01:32:07 Downloading - Downloading input data
2022-06-27 01:32:07 Training - Downloading the training image...........................
2022-06-27 01:36:24 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-06-27 01:36:27,285 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-06-27 01:36:27,325 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-06-27 01:36:27,334 sagemaker_pytorch_container.training INFO     Invoking user training script.
2022-06-27 01:36:27,780 sagemaker-training-toolkit INFO     Installing dependencies from requirements.txt:
/opt/conda/bin/python3.8 -m pip install -r requirements.txt
bash: cannot set te

In [ ]:
hyperparameter_ranges = {
    "lr": CategoricalParameter([3e-4, 1e-4, 5e-5, 3e-5]),
    "batch-size": CategoricalParameter([4, 8, 16]),
}

# change to accuracy
objective_metric_name = "average test loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "average test loss", "Regex": "Test set: Average loss: ([0-9\\.]+)"}]

In [ ]:
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=8,
    max_parallel_jobs=2,
    objective_type=objective_type,
)

In [ ]:
tuner.fit({"training": inputs})